![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=TechnologyStudies/ComputingScience/Courses/procedural-programming-1.ipynb&depth=1"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# CSE2110: Procedural Programming 1

*[Alberta Education Learning Outcomes-Business, Administration, Finance & Information Technology (BIT)](https://education.alberta.ca/media/159479/cse_pos.pdf)*

*Computer Science-Page 5*


*Prerequisite: [CSE1120: Structured Programming 2](structured-programming-2.ipynb)*

***

Students develop their understanding of the procedural programming paradigm. They move from a structured programming approach in which modules were handled through the use of program blocks to a more formal modular programming approach in which they are handled through subprograms. In the process, students also learn to use a number of new design approaches made possible by the new paradigms. As part of this process, they also learn what types of problems are amenable to modular algorithms and programs.



## APIs and Modules

We are going to explore the use of the [Requests](https://docs.python-requests.org/en/latest/index.html) library with an [API](https://en.wikipedia.org/wiki/API) (Application Programming Interface) to get data that we can analyse.

For more information, check out [this video about APIs](https://www.youtube.com/watch?v=OVvTv9Hy91Q).

Since the required code is somewhat long, we are going to import it from a module (a `.py` file stored in the [same folder as this notebook](.)).

There are two different APIs for you to choose from, either Pokémon ([PokéAPI](https://pokeapi.co)) or Music ([Spotify API](https://developer.spotify.com/documentation/web-api)).

### Option A: PokéAPI

We'll define a function to get data from PokéAPI. Since we are getting data about all of the Pokémon this will take a few minutes to run.

In [ ]:
import requests
import pandas as pd

def get_pokemon_data():
    try:
        r = requests.get('https://pokeapi.co/api/v2/pokemon?limit=100000')
        df = pd.DataFrame(r.json()['results'])

        abilities = []
        base_experience = []
        forms = []
        #game_indices = []
        height = []
        held_items = []
        id = []
        is_default = []
        location_area_encounters = []
        moves = []
        #name = []
        order = []
        species = []
        sprites = []
        stats = []
        types = []
        weight = []
        for i in range(len(df)):
            print(df['name'][i])
            try:
                r = requests.get(df['url'][i])
                abilities.append(r.json()['abilities'])
                base_experience.append(r.json()['base_experience'])
                forms.append(r.json()['forms'])
                #game_indices.append(r.json()['game_indices'])
                height.append(r.json()['height'])
                held_items.append(r.json()['held_items'])
                id.append(r.json()['id'])
                is_default.append(r.json()['is_default'])
                location_area_encounters.append(r.json()['location_area_encounters'])
                moves.append(r.json()['moves'])
                #name.append(r.json()['name'])
                order.append(r.json()['order'])
                species.append(r.json()['species'])
                sprites.append(r.json()['sprites'])
                stats.append(r.json()['stats'])
                types.append(r.json()['types'])
                weight.append(r.json()['weight'])
            except:
                abilities.append(None)
                base_experience.append(None)
                forms.append(None)
                #game_indices.append(None)
                height.append(None)
                held_items.append(None)
                id.append(None)
                is_default.append(None)
                location_area_encounters.append(None)
                moves.append(None)
                #name.append(None)
                order.append(None)
                species.append(None)
                sprites.append(None)
                stats.append(None)
                types.append(None)
                weight.append(None)
        df['id'] = id
        df['height'] = height
        df['weight'] = weight
        df['base_experience'] = base_experience
        df['abilities'] = abilities
        df['forms'] = forms
        df['species'] = species
        df['is_default'] = is_default
        #df['game_indices'] = game_indices
        #df['held_items'] = held_items
        df['location_area_encounters'] = location_area_encounters
        #df['moves'] = moves
        #df['name'] = name
        df['order'] = order
        #df['sprites'] = sprites
        df['stats'] = stats
        df['types'] = types
        df = df.drop('url', axis=1) # drop url column

        # split the stats column into multiple columns
        df_stats = pd.DataFrame(df['stats'].tolist())
        df_stats.columns = ['hp','attack','defense','special-attack','special-defense','speed']
        for column in df_stats.columns:
            df_stats[column] = df_stats[column].apply(lambda x: x['base_stat'] if x is not None else None)
        # join with the main dataframe
        df = df.join(df_stats)
        df = df.drop('stats', axis=1)
        df = df.drop('location_area_encounters', axis=1)

        # split the abilities to three columns
        abilities_lists = [[],[],[]]
        for i in range(len(df)):
            for n in range(3):
                try:
                    ability = df['abilities'][i][n]['ability']['name']
                except:
                    ability = None
                abilities_lists[n].append(ability)
        df['ability1'] = abilities_lists[0]
        df['ability2'] = abilities_lists[1]
        df['ability3'] = abilities_lists[2]
        df = df.drop('abilities', axis=1)

        # species column is a dictionary, so we need to extract the name
        df['species'] = df['species'].apply(lambda x: x['name'] if x is not None else None)

        # convert the forms column to a string
        df['forms'] = df['forms'].apply(lambda x: x[0]['name'] if x is not None else None)

        # split the types to two columns
        types_lists = [[],[]]
        for i in range(len(df)):
            for n in range(2):
                try:
                    type = df['types'][i][n]['type']['name']
                except:
                    type = None
                types_lists[n].append(type)
        df['type1'] = types_lists[0]
        df['type2'] = types_lists[1]
        df = df.drop('types', axis=1)

        # convert the moves column to a stringified list
        #df['moves'] = df['moves'].apply(lambda x: [move['move']['name'] for move in x] if x is not None else None)

        # reorder the columns
        df = df[['id','name','base_experience','height','weight','speed','hp','attack','defense','special-attack','special-defense','forms','species','is_default','order','type1','type2','ability1','ability2','ability3']]

    except:
        # if something didn't work, then just read the existing csv file
        df = pd.read_csv('data/pokemon.csv')
    
    # return the dataframe
    return df

# get the data
pokemon = get_pokemon_data()

# write the data to a csv file
pokemon.to_csv('pokemon.csv', index=False)

pokemon

We can also define a function to get a link to the an image of a Pokémon, then display it with `Image(sprite_url)`.

In [ ]:
from IPython.display import Image

def get_sprite_url(id):
    image_url = 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/other/official-artwork/'+str(id)+'.png'
    return image_url

sprite_url = get_sprite_url(3)
display(Image(sprite_url))

### Option B: Spotify API

The Spotify Web API allows us to get information about songs, albums, and artists. If you want to retireve more data and have a [Spotify account](https://www.spotify.com/us/signup), you can sign in to the [Developers Dashboard](https://developer.spotify.com/dashboard/login).

From the Dashboard, you can click the `CREATE AN APP` button, type a name and description, and then click `CREATE`. Clicking on your new app in the Dashboard will show you the `Client ID` and `CLIENT SECRET` that you can paste into the code cell below.

The code cell below will define some functions to get data about music tracks from the Spotify API.

In [ ]:
CLIENT_ID = 'PASTE_YOUR_CLIENT_ID_HERE'
CLIENT_SECRET = 'PASTE_YOUR_CLIENT_SECRET_HERE'

import requests
import pandas as pd

def spotify_setup(CLIENT_ID, CLIENT_SECRET):
    try:
        auth_response = requests.post('https://accounts.spotify.com/api/token', {'grant_type':'client_credentials', 'client_id':CLIENT_ID, 'client_secret':CLIENT_SECRET})
        auth_response_data = auth_response.json()
        access_token = auth_response_data['access_token']
        global headers
        headers = {'Authorization':'Bearer {token}'.format(token=access_token)}
        print('Setup complete')
    except:
        print('Remember to paste your client ID and secret into the code')

def find_tracks(search_string):
    try:
        r = requests.get('https://api.spotify.com/v1/search?q=' + search_string + '&type=track', headers=headers)
        info = r.json()
    except:
        print('Error with search string:', search_string)
        print('Did you run spotify_setup(CLIENT_ID, CLIENT_SECRET) first?')
        info = None
    return info

def get_track_info(track_id):
    try:
        r = requests.get('https://api.spotify.com/v1/tracks/' + track_id, headers=headers)
        info = r.json()
    except:
        print('Error with track id:', track_id)
        print('Did you run spotify_setup(CLIENT_ID, CLIENT_SECRET) first?')
        info = None
    return info

def get_track_features(track_id):
    try:
        r = requests.get('https://api.spotify.com/v1/audio-features/' + track_id, headers=headers)
        info = r.json()
    except:
        print('Error with track id:', track_id)
        print('Did you run spotify_setup(CLIENT_ID, CLIENT_SECRET) first?')
        info = None
    return info

def get_playlist(playlist_id):
    tracks = []
    for x in range(50):  
        offset = x*100  # it only returns 100 tracks at a time
        try:
            r = requests.get('https://api.spotify.com/v1/playlists/' + playlist_id + '/tracks?offset=' + str(offset), headers=headers)
            for item in r.json()['items']:
                tracks.append([item['track']['artists'][0]['name'], item['track']['name'], item['track']['id'], item['track']['album']['release_date']])
        except:
            print(offset)
            break
    pl = pd.DataFrame(tracks, columns=['artist', 'track', 'id', 'release_date'])

    track_features = {}
    for row in pl.itertuples():
        print(row[1], row[2]) # artist and track
        id = row[3]
        features = get_track_features(id)
        track_features[id] = features

    from IPython.display import clear_output
    clear_output()

    tf = pd.DataFrame(track_features).T
    playlist = pd.merge(pl, tf, on='id')
    return playlist

spotify_setup(CLIENT_ID, CLIENT_SECRET)

To get a data set from playlist, either one you created or someone else's playlist, you need the playlist ID [from the playlist link](https://clients.caster.fm/knowledgebase/110/How-to-find-Spotify-playlist-ID.html).

In [ ]:
playlist_id = '37i9dQZF1DX7iB3RCnBnN4'

playlist = get_playlist(playlist_id)
playlist

---
By double clicking on this cell below, you will be able to add your personal reflections and comments.

1. Which of the following are advantages of programming with modules or subroutines like this?

    a. reducing the duplication of code in a program

    b. enabling the reuse of code in more than one program
    
    c. decomposing complex problems into simpler pieces to improve maintainability and extendibility
    
    d. improving the readability of a program
    
    e. hiding or protecting the program data

    f. all of the above

2. Look through the Python code in the [get_pokemon.py](get_pokemon.py) or the [get_spotify.py](get_spotify.py) and answer the following questions.

    i. Python supports [functions](https://www.w3schools.com/python/python_functions.asp), [lambda functions](https://www.w3schools.com/python/python_lambda.asp), [classes](https://www.w3schools.com/python/python_classes.asp), and [generators](https://wiki.python.org/moin/Generators). Which of those do you see in the `.py` file you looked at?

    ii. How is using one of these files similar to and different from using a built-in library such as [statistics](https://docs.python.org/3/library/statistics.html) or someone else's library such as [Requests](https://docs.python-requests.org/en/latest/index.html)?

    iii. What is the purpose of using the `try:` and `except:` lines in the code? What would happen if they were not included?

    iv. What does a `return` line do in Python?

    v. What is a [parameter or argument](https://www.w3schools.com/python/gloss_python_function_arguments.asp) in Python? Give an example of how that is used in [get_pokemon.py](get_pokemon.py) or [get_spotify.py](get_spotify.py)

## Personal Connections and Career Pathways

3. What are some of your interests, values, beliefs, resources, prior learning, and experiences that relate to what you are learning about computing science?

4. Write a paragraph about how the knowledge and skills you are learning learning could relate to your future career choices.

1. 


2. 


3. 


4. 



---
## Data Science

Now that you have data in `pokemon` or `playlist`, use what you learned in [Structured Programming 1](structured-programming-1.ipynb) and [Structured Programming 2](structured-programming-2.ipynb) to find and communicate insights from the data.

Add more code and markdown cells as necessary. You must include at least two different visualizations, and text in markdown cells to communicate what your code is doing as well as the insights you are extracting.

Now that you have completed this notebook, submit your work to your teacher by downloading this notebook.

This can be done in the following ways:

- .ipynb extension
- .html
- .pdf


---

# Outcomes:

The student will:

1. demonstrate an understanding of modular programming
    1. describe the advantages of programming with modules or subroutines including:
        1. reducing the duplication of code in a program
        2. enabling the reuse of code in more than one program
        3. decomposing complex problems into simpler pieces to improve maintainability and extendibility
        4. improving the readability of a program
        5. hiding or protecting the program data
    2. select a programming environment and describe how it supports procedural programming including:
        1. the type of subprograms supported; e.g., procedures, functions, methods
        2. the level or type of modularity provided
        3. the level of protection provided from unwanted side-effects
<br><br>
2. demonstrate basic procedural programming skills by writing algorithms employing a modular approach to solve problems
    1. analyze a data processing problem and use a top-down design approach to decompose it into discreet input, processing and output modules
    2. analyze and refine modules into submodules that are a manageable size for each process; e.g., input submodules, processing submodules and output submodules
    3. describe and represent, using pseudocode or an appropriate diagramming approach, the relationship among the modules
    4. analyze and rewrite algorithms for each module identifying the pre- and post-conditions and required program control of flow mechanisms.
    5. analyze and evaluate algorithms for each developing module with appropriate data and revise, as required
<br><br>
3. translate algorithms into source code, convert the source code into machine executable form, execute and debug, as required
    1. convert an algorithm into a program of linked subprograms with a main or client module calling other modules in a manner that reflects the structure of the algorithm
    2. use appropriate types of subprograms to implement the various sections of the algorithm; e.g., functions (subprograms that return a value) and procedures (subprograms that do not return a value)
    3. analyze and determine the type of scope required to protect and/or hide data and keep implementation decoupled from the calling modules and to avoid unwanted side-effects with consideration to:
        1. use of appropriate parameters for importing and exporting data to and from subprograms
        2. use of local variables and nested subprograms to enhance cohesion
        3. one- and two-way parameter passing for importing and exporting data to and from subprograms
    4. analyze for, and maintain, an appropriate balance between the coupling or dependency and cohesion or focus of subprograms
    5. create both internal and external documentation
    6. analyze the program and eliminate syntax, run-time and logic errors by using appropriate test data for each programming task at each stage of development
<br><br>
4. compare the results of the program with the intent of the algorithm and modify, as required
    1. use appropriate error trapping mechanisms built into the programming environment, as well as programmer-directed error-trapping techniques, to eliminate logic errors and debug the program
    2. compare the congruency between the outcomes of the debugged program and the original intent of the algorithm and modify, as required
<br><br>
5. demonstrate basic competencies
    1. demonstrate fundamental skills to:
        1. communicate
        2. manage information
        3. use numbers
        4. think and solve problems
    2. demonstrate personal management skills to:
        1. demonstrate positive attitudes and behaviours
        2. be responsible
        3. be adaptable
        4. learn continuously
        5. work safely
    3. demonstrate teamwork skills to:
        1. work with others
        2. participate in projects and tasks
<br><br>
6. identify possible life roles related to the skills and content of this cluster
    1. recognize and then analyze the opportunities and barriers in the immediate environment
    2. identify potential resources to minimize barriers and maximize opportunities

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)